In [1]:
!pip install langchain_community[all]
!pip install llama-parse
!pip install qdrant_client
!pip install langchain_qdrant
!pip install ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


Import Library



In [1]:
import os
import uuid
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from langchain_qdrant import FastEmbedSparse, Qdrant
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document
import nest_asyncio
from llama_parse import LlamaParse
import ollama
from langchain_community.embeddings import OllamaEmbeddings

c:\Users\ASUS-3070\Desktop\Github\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nest_asyncio.apply()

API

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
qdrant_url = os.environ["QDRANT_URL"]
qdrant_api_key = os.environ["QDRANT_API_KEY"]
collection_name = "Ollama-RAG"

qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

In [4]:
# --- Vectorstore Initialization (Langchain Qdrant) ---
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=OllamaEmbeddings,  # We're handling embeddings separately
)

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\3637584716.py:2: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(
c:\Users\ASUS-3070\Desktop\Github\.venv\lib\site-packages\langchain_community\vectorstores\qdrant.py:152: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


Loading and Embedded Function


In [7]:
def load_and_embed_documents(file_path):
    def load_document(file_path):
        if file_path.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.endswith(".txt"):
            loader = TextLoader(file_path)
        elif file_path.endswith(".md"):
            loader = UnstructuredMarkdownLoader(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_path}")
        documents = loader.load()
        return documents

    def create_chunks(documents):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
        chunks = text_splitter.split_documents(documents)
        return chunks

    documents = load_document(file_path)
    chunks = create_chunks(documents)

    # Embeddings setup:
    sparse_embeddings_engine = FastEmbedSparse(model_name="Qdrant/bm25")

    for chunk in chunks:
        text = chunk.page_content
        dense_embedding = ollama.embeddings(model='mxbai-embed-large', prompt=text)['embedding']
        sparse_embedding = sparse_embeddings_engine.embed_query(text)

        # Create a Langchain Document object with dense and sparse vectors
        doc = Document(
            page_content=text,
            metadata=chunk.metadata,
            embedding={"dense": dense_embedding, "sparse": sparse_embedding}
        )

        # Add the document to the Langchain vectorstore
        vectorstore.add_documents([doc])

Hybrid Search Function

In [5]:
def hybrid_search(query: str, collection: str, limit: int = 5):
    dense_vector = ollama.embeddings(model='rjmalagon/gte-qwen2-1.5b-instruct-embed-f16', prompt=query)

    results = qdrant_client.search(
            collection_name=collection_name,
            query_vector=models.NamedVector(
                name="gte-qwen1.5",
                vector=dense_vector['embedding']
            ),
            limit=limit,
            search_params=models.SearchParams(
                hnsw_ef=128
            )
    )
    return results

In [6]:
hybrid_search("What is machine learning?", "Ollama-RAG")

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


[ScoredPoint(id=8, version=0, score=0.62523055, payload={'text': '1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.', 'metadata': {'source': 'C:/Users/shaoe/OneDrive/Desktop/AAI3008LLM/The_Hundred_page_Machine_Learning_Book_Andriy_Burkov_Z_Library.pdf', 'page': 2}}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=4, version=0, score=0.5865381, payload={'text': 'As you can see, just like artiﬁcial intelligence is not intelligence, machine learning is not\nlearning. However, machine learning is a universally recognized term that usu

In [7]:
def retrieve_documents(query):
    results = hybrid_search(query, collection=collection_name, limit=3)  # Use hybrid search here
    list = "Database Context: "
    for i in range(len(results)):
        list += str(results[i].payload['text'])
    return list

retrieve_documents("What is machine learning?")

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


'Database Context: 1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.As you can see, just like artiﬁcial intelligence is not intelligence, machine learning is not\nlearning. However, machine learning is a universally recognized term that usually refers\nto the science and engineering of building machines capable of doing various useful things\nwithout being explicitly programmed to do so. So, the word “learning” in the term is used\nby analogy with the learning in animals rather than literally.\nWho This Book is ForPreface\nLet’s start by telling 

RAG Chain

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import GoogleSearchAPIWrapper

In [9]:
def get_rag_chain(enable_search=False):
    """Creates and returns the RAG chain with hybrid search and optional Google Search."""

    # Initialize the LLM
    llm = Ollama(model="phi3:mini")

    # Create a prompt template
    template = """Answer the question based on the following context:
    {context}
    {online_context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Function to retrieve relevant documents using hybrid search
    def retrieve_documents(query):
        results = hybrid_search(query, collection=collection_name, limit=3)  # Upgrade to hybrid search here
        list = "Database Context: "
        for i in range(len(results)):
            list += str(results[i].payload['text'])
        #print(f"{list}") # This one maybe fix to add metadata content
        return list 

    # Add online searching, defaulting to empty string if search is disabled
    def online_search(query):
        """Searches Google and returns a summary of the results."""
        search = GoogleSearchAPIWrapper()
        search_results = search.run(query)
        if not search_results:
            search_results = "No relevant information found online."  # Handle empty results
        #print(f"Online Search Results: {search_results}")  # Debugging
        return f"Online search results: {search_results}"
    
    def filler(query):
        #print("Online Search Switched off")
        return f"Online Search Switched Off"
    
    if enable_search:
        rag_chain = (
        {"context": retrieve_documents, ### This need 
         "question": RunnablePassthrough(),
         "online_context": online_search} ### This is correct
        | prompt
        | llm
        | StrOutputParser()
    )
    else:
        rag_chain = (
        {"context": retrieve_documents, ### This need 
         "question": RunnablePassthrough(),
         "online_context": filler} ### This is correct
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

Search Tool

In [10]:
def agent_search(query):
    """Search for relevant information in the document and online."""
    rag_chain = get_rag_chain(enable_search=False)  # Enable online search
    response = rag_chain.invoke(query)
    return response

In [11]:
def load_user_document(file_path):
    """Loads a user-provided document and adds it to the vectorstore."""
    try:
        load_and_embed_documents(file_path)
        return "Document loaded and processed successfully."
    except Exception as e:
        return f"Error loading document: {e}"

In [12]:
from langchain.agents import Tool

In [13]:
tools = [
    Tool(
        name="document_search",
        func=agent_search,
        description="useful for when you need to answer questions about the document and online."
    )
]

In [14]:
# --- Agent Initialization ---
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [15]:
llm = Ollama(model="phi3:mini")
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\1301706811.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi3:mini")
C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\1301706811.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://pytho

In [16]:
from langchain_google_community import GoogleSearchAPIWrapper

In [18]:
query = "What is machine learning?"
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
Question: What is machine learning?
Thought: To provide a comprehensive definition of machine learning, it would be beneficial to consult recent and reliable sources. I will use the document_search tool for this purpose.
Action: document_search
Action Input: "What is machine learning?"

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(



Observation: Machine Learning is an integral subfield of computer science, primarily focused on designing and implementing algorithms that can solve practical problems effectively by gathering a dataset and algorithmically constructing statistical models based on it. These solutions are useful as they do not rely explicitly programmed methods but rather derive mathematical formulas capable of producing desired outputs when applied to collections of inputs known as "training data." While these systems might also perform well for most other unseen inputs, this largely depends if those come from the same or a similar statistical distribution as the training dataset. In essence, while machines don't learn in the human sense, they find patterns and infer rules that mimic learning by analogy with animal behavior rather than genuine understanding of data manipulation.
Thought:Question: What is machine learning?
Thought: To provide a comprehensive definition of machine learning, it would be b

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_10464\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(



Observation: Machine Learning, often abbreviated as ML, is a subfield of computer science that focuses on building algorithms capable of discovering patterns and making predictions based on data without being explicitly programmed. These algorithms rely heavily on examples or datasets to learn from them through an iterative process known as training. The goal of machine learning methods typically involves constructing statistical models in order to automate analytical model building, thus facilitating problem-solving tasks such as classification and prediction which usually require insight into the system being studied. While machines don't "learn" like animals do, they perform a process by identifying mathematical relationships within data that can help them predict future outcomes or classify inputs based on past experiences—similar to how humans learn through practice and repetition of tasks. In essence, machine learning combines statistical techniques with algorithms designed to a